In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
def get_model():
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [ ]:
model = get_model()

In [ ]:
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

model.save('my_model')

reconstructed_model = keras.models.load_model('my_model')

np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

reconstructed_model.fit(test_input, test_target)

In [ ]:
!ls my_model

In [ ]:
class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        self.hidden_units = hidden_units
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]
    
    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x
    
    def get_config(self):
        return {'hidden_units': self.hidden_units}
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

model = CustomModel([16, 16, 10])

input_arr = tf.random.uniform((1, 5))

outputs = model(input_arr)

model.save('my_model')

In [ ]:
loaded_1 = keras.models.load_model('my_model', custom_objects={'CustomModel': CustomModel})

del CustomModel

loaded_2 = keras.models.load_model('my_model')

np.testing.assert_allclose(loaded_1(input_arr), outputs)
np.testing.assert_allclose(loaded_2(input_arr), outputs)

print(model)
print(loaded_1)
print(loaded_2)

In [ ]:
model = get_model()

test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

model.save('my_h5_model.h5')

reconstructed_model = keras.models.load_model('my_h5_model.h5')

np.testing.assert_allclose(model.predict(test_input), reconstructed_model.predict(test_input))

reconstructed_model.fit(test_input, test_target)

In [ ]:
layer = keras.layers.Dense(3, activation='relu')
layer_config = layer.get_config()
new_layer = keras.layers.Dense.from_config(layer_config)

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(32,)),
    keras.layers.Dense(1)
])

config = model.get_config()
new_model = keras.Sequential.from_config(config)

In [ ]:
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)

config = model.get_config()
new_model = keras.Model.from_config(config)

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(32,)),
    keras.layers.Dense(1)
])

json_config = model.to_json()
new_model = keras.models.model_from_json(json_config)

In [ ]:
model.save('my_model')
tensorflow_graph = tf.saved_model.load('my_model')
x = np.random.uniform(size=(4, 32)).astype('float32')
predicted = tensorflow_graph(x)

In [ ]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name='var_a')
    
    def call(self, inputs, training=False):
        if training:
            return inputs * self.var
        else:
            return inputs
    
    def get_config(self):
        return {'a': self.var.numpy()}
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

layer = CustomLayer(5)
layer.var.assign(2)

serialized_layer = keras.layers.serialize(layer)
new_layer = keras.layers.deserialize(serialized_layer, custom_objects = {'CustomLayer': CustomLayer})

In [ ]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer='random_normal', trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({'units': self.units})
        return config
    
def custom_activation(x):
    return tf.nn.tanh(x) ** 2

inputs = keras.Input((32, ))
x = CustomLayer(32)(inputs)
outputs = keras.layers.Activation(custom_activation)(x)
model = keras.Model(inputs, outputs)

config = model.get_config()

custom_objects = {'CustomLayer': CustomLayer, 'custom_activation': custom_activation}

with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.Model.from_config(config)

In [ ]:
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.models.clone_model(model)

In [ ]:
def create_layer():
    layer = keras.layers.Dense(64, activation='relu', name='dense_2')
    layer.build((None, 784))
    return layer

layer_1 = create_layer()
layer_2 = create_layer()

layer_2.set_weights(layer_1.get_weights())

In [ ]:
inputs = keras.Input((784,), name='digits')
x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = keras.layers.Dense(10, name='predictions')(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name='2_layer_mlp')

class SubclassedModel(keras.Model):
    def __init__(self, output_dim=10, name=None):
        super(SubclassedModel, self).__init__(name=name)
        self.output_dim = output_dim
        self.dense_1 = keras.layers.Dense(64, activation='relu', name='dense_1')
        self.dense_2 = keras.layers.Dense(64, activation='relu', name='dense_2')
        self.dense_3 = keras.layers.Dense(self.output_dim, name='predictions')
    
    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return x
    
    def get_config(self):
        return {'output_dim': self.output_dim, 'name': self.name}

subclassed_model = SubclassedModel(10)
subclassed_model(tf.ones((1, 784)))

subclassed_model.set_weights(functional_model.get_weights())
assert len(subclassed_model.weights) == len(functional_model.weights)

for a, b in zip(functional_model.weights, subclassed_model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

In [ ]:
inputs = keras.Input(shape=(784,), name='digits')
x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = keras.layers.Dense(10, name='predictions')(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')

inputs = keras.Input(shape=(784,), name='digits')
x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, name='predictions')(x)
functional_model_with_dropout = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')

functional_model_with_dropout.set_weights(functional_model.get_weights())

In [ ]:
sequential_model = keras.Sequential([
    keras.Input(shape=(784,), name='digits'),
    keras.layers.Dense(64, activation='relu', name='dense_1'),
    keras.layers.Dense(64, activation='relu', name='dense_2'),
    keras.layers.Dense(10, name='predictions')
])

sequential_model.save_weights('ckpt')
load_status = sequential_model.load_weights('ckpt')

load_status.assert_consumed()

In [ ]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name='var_a')
    
layer = CustomLayer(5)
layer_checkpoint = tf.train.Checkpoint(layer=layer).save('custom_layer')
ckpt_reader = tf.train.load_checkpoint(layer_checkpoint)
ckpt_reader.get_variable_to_dtype_map()


In [ ]:
inputs = keras.Input(shape=(784,), name='digits')
x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = keras.layers.Dense(10, name='predictions')(x)
functional_model = keras.Model(inputs, outputs, name='3_layer_mlp')

pretrained_model = keras.Model(functional_model.inputs, functional_model.layers[-1].input, name='pretrained_model')
for w in pretrained_model.weights:
    w.assign(tf.random.normal(w.shape))
pretrained_model.save_weights('pretrained_ckpt')
pretrained_model.summary()

In [ ]:
inputs = keras.Input(shape=(784, ), name='digits')
x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = keras.layers.Dense(5, name='predictions')(x)
model = keras.Model(inputs, outputs, name='new_model')

model.load_weights('pretrained_ckpt')

for a, b in zip(pretrained_model.weights, model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

print('\n', '_' * 50)
model.summary()

In [ ]:
inputs = keras.Input(shape=(784, ), name='digits')
x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
pretrained_model = keras.Model(inputs, x, name='pretrained')

model = keras.Sequential([
    pretrained_model,
    keras.layers.Dense(5, name='predictions')
])

model.summary()

In [ ]:
pretrained_model.load_weights('pretrained_ckpt')

In [ ]:
first_dense = functional_model.layers[1]
last_dense = functional_model.layers[-1]

ckpt_path = tf.train.Checkpoint(dense=first_dense, kernel=last_dense.kernel, bias=last_dense.bias).save('ckpt')

class ContrivedModel(keras.Model):
    def __init__(self):
        super(ContrivedModel, self).__init__()
        self.first_dense = keras.layers.Dense(64)
        self.kernel = self.add_variable('kernel', shape=(64, 10))
        self.bias = self.add_variable('bias', shape=(10, ))
    
    def call(self, inputs):
        x = self.first_dense(inputs)
        return tf.matmul(x, self.kernel) + self.bias

model = ContrivedModel()
_ = model(tf.ones((1, 784)))

tf.train.Checkpoint(dense=model.first_dense, kernel=model.kernel, bias=model.bias).restore(ckpt_path).assert_consumed()

In [ ]:
sequential_model = keras.Sequential([
    keras.Input(shape=(784, ), name='digits'),
    keras.layers.Dense(64, activation='relu', name='dense_1'),
    keras.layers.Dense(64, activation='relu', name='dense_2'),
    keras.layers.Dense(10, name='predictions')
])

sequential_model.save_weights('weights.h5')
sequential_model.load_weights('weights.h5')

In [ ]:
class NestedDenseLayer(keras.layers.Layer):
    def __init__(self, units, name=None):
        super(NestedDenseLayer, self).__init__(name=name)
        self.dense_1 = keras.layers.Dense(units, name='dense_1')
        self.dense_2 = keras.layers.Dense(units, name='dense_2')
    
    def call(self, inputs):
        return self.dense_2(self.dense_1(inputs))

nested_model = keras.Sequential([
    keras.Input(shape=(784,), name='digits'),
    NestedDenseLayer(10, 'nested'),
])
variable_names = [v.name for v in nested_model.weights]
print('Variables: {}'.format(variable_names))

In [ ]:
print("\nChanging trainable status of one of the nested layers...")
nested_model.get_layer('nested').dense_1.trainable = False
variable_names_2 = [v.name for v in nested_model.weights]
print('Variables: {}'.format(variable_names_2))
print('Variable ordering changed: ', variable_names != variable_names_2)

In [ ]:
def create_functional_model():
    inputs = keras.Input(shape=(784, ), name='digits')
    x = keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
    x = keras.layers.Dense(64, activation='relu', name='dense_2')(x)
    outputs = keras.layers.Dense(10, name='predictions')(x)
    return keras.Model(inputs, outputs)

functional_model = create_functional_model()
functional_model.save_weights('pretrained_weights.h5')

pretrained_model = create_functional_model()
pretrained_model.load_weights('pretrained_weights.h5')

extracted_layers = pretrained_model.layers[:-1]
extracted_layers.append(keras.layers.Dense(5, name='dense_3'))
model = keras.Sequential(extracted_layers)
model.summary()